## Recommendations with MovieTweetings: Collaborative Filtering

One of the most popular methods for making recommendations is **collaborative filtering**.  In collaborative filtering, you are using the collaboration of user-item recommendations to assist in making new recommendations.  

There are two main methods of performing collaborative filtering:

1. **Neighborhood-Based Collaborative Filtering**, which is based on the idea that we can either correlate items that are similar to provide recommendations or we can correlate users to one another to provide recommendations.

2. **Model Based Collaborative Filtering**, which is based on the idea that we can use machine learning and other mathematical models to understand the relationships that exist amongst items and users to predict ratings and provide ratings.


In this notebook, you will be working on performing **neighborhood-based collaborative filtering**.  There are two main methods for performing collaborative filtering:

1. **User-based collaborative filtering:** In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

2. **Item-based collaborative filtering:** In this type of recommendation, first you need to find the items that are most related to each other item (based on similar ratings).  Then you can use the ratings of an individual on those similar items to understand if a user will like the new item.

In this notebook you will be implementing **user-based collaborative filtering**.  However, it is easy to extend this approach to make recommendations using **item-based collaborative filtering**.  First, let's read in our data and necessary libraries.

**NOTE**: Because of the size of the datasets, some of your code cells here will take a while to execute, so be patient!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import ast
import tests as t
from scipy.sparse import csr_matrix
from IPython.display import HTML

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

print(reviews.head())

   user_id  movie_id  rating   timestamp                 date
0        1    114508       8  1381006850  2013-10-05 21:00:50
1        2    208092       5  1586466072  2020-04-09 21:01:12
2        2    358273       9  1579057827  2020-01-15 03:10:27
3        2  10039344       5  1578603053  2020-01-09 20:50:53
4        2   6751668       9  1578955697  2020-01-13 22:48:17


### Measures of Similarity

When using **neighborhood** based collaborative filtering, it is important to understand how to measure the similarity of users or items to one another.  

There are a number of ways in which we might measure the similarity between two vectors (which might be two users or two items).  In this notebook, we will look specifically at two measures used to compare vectors:

* **Pearson's correlation coefficient**

Pearson's correlation coefficient is a measure of the strength and direction of a linear relationship. The value for this coefficient is a value between -1 and 1 where -1 indicates a strong, negative linear relationship and 1 indicates a strong, positive linear relationship. 

If we have two vectors x and y, we can define the correlation between the vectors as:


$$CORR(x, y) = \frac{\text{COV}(x, y)}{\text{STDEV}(x)\text{ }\text{STDEV}(y)}$$

where 

$$\text{STDEV}(x) = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}$$

and 

$$\text{COV}(x, y) = \frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

where n is the length of the vector, which must be the same for both x and y and $\bar{x}$ is the mean of the observations in the vector.  

We can use the correlation coefficient to indicate how alike two vectors are to one another, where the closer to 1 the coefficient, the more alike the vectors are to one another.  There are some potential downsides to using this metric as a measure of similarity.  You will see some of these throughout this workbook.


* **Euclidean distance**

Euclidean distance is a measure of the straightline distance from one vector to another.  Because this is a measure of distance, larger values are an indication that two vectors are different from one another (which is different than Pearson's correlation coefficient).

Specifically, the euclidean distance between two vectors x and y is measured as:

$$ \text{EUCL}(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

Different from the correlation coefficient, no scaling is performed in the denominator.  Therefore, you need to make sure all of your data are on the same scale when using this metric.

**Note:** Because measuring similarity is often based on looking at the distance between vectors, it is important in these cases to scale your data or to have all data be in the same scale.  In this case, we will not need to scale data because they are all on a 10 point scale, but it is always something to keep in mind!

------------

### User-Item Matrix

In order to calculate the similarities, it is common to put values in a matrix.  In this matrix, users are identified by each row, and items are represented by columns.  


![alt text](images/userxitem.png "User Item Matrix")


In the above matrix, you can see that **User 1** and **User 2** both used **Item 1**, and **User 2**, **User 3**, and **User 4** all used **Item 2**.  However, there are also a large number of missing values in the matrix for users who haven't used a particular item.  A matrix with many missing values (like the one above) is considered **sparse**.

Our first goal for this notebook is to create the above matrix with the **reviews** dataset.  However, instead of 1 values in each cell, you should have the actual rating.  

The users will indicate the rows, and the movies will exist across the columns. To create the user-item matrix, we only need the first three columns of the **reviews** dataframe, which you can see by running the cell below.

In [2]:
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_items.head()

,user_id,movie_id,rating
0,1,114508,8
1,2,208092,5
2,2,358273,9
3,2,10039344,5
4,2,6751668,9


### Creating the User-Item Matrix

In order to create the user-items matrix (like the one above), I personally started by using a [pivot table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html). 

However, I quickly ran into a memory error (a common theme throughout this notebook).  I will help you navigate around many of the errors I had, and achieve useful collaborative filtering results! 

_____

`1.` Create a matrix where the users are the rows, the movies are the columns, and the ratings exist in each cell, or a NaN exists in cells where a user hasn't rated a particular movie. If you get a memory error (like I did), [this link here](https://stackoverflow.com/questions/39648991/pandas-dataframe-pivot-memory-error) might help you!

In [4]:
# Create user-by-item matrix
matrix = pd.pivot_table(user_items, values = 'rating', index = 'user_id', columns= 'movie_id')

In [5]:
matrix

movie_id,91,417,3863,4101,4210,4395,4413,4546,4972,5074,...,11358398,11388406,11388580,11390036,11454170,11561866,11642302,11644170,11777040,12131262
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [7]:
user_by_movie

movie_id,91,417,3863,4101,4210,4395,4413,4546,4972,5074,...,11358398,11388406,11388580,11390036,11454170,11561866,11642302,11644170,11777040,12131262
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check your results below to make sure your matrix is ready for the upcoming sections.

In [8]:
assert reviews.user_id.nunique() == matrix.shape[0], "Oh no! Your matrix should have {} rows, and yours has {}!".format(reviews.user_id.nunique(), user_by_movie.shape[0])
print("Looks like you are all set! Proceed!")
HTML('<img src="images/greatjob.webp">')

Looks like you are all set! Proceed!


`2.` Now that you have a matrix of users by movies, use this matrix to create a dictionary where the key is each user and the value is an array of the movies each user has rated.

In [11]:
user_by_movie.iloc[2][user_by_movie.loc[2].isnull() == False].index.values

array([  208092,   358273,  6751668,  7131622,  7975244,  7984734,
        8367814,  8579674,  8946378, 10039344], dtype=int64)

In [14]:
# Create a dictionary with users and corresponding movies seen

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    # Implement your code here
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values
    return movies


def create_user_movie_dict():
    '''
    INPUT: None
    OUTPUT: movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    Creates the movies_seen dictionary
    '''
    # Implement your code here
    movies_seen = dict()
    n_users = user_by_movie.shape[0]
    for user1 in range(1, n_users+1):
        movies_seen[user1] = movies_watched(user1)
    return movies_seen

movies_seen = create_user_movie_dict()

In [15]:
movies_seen

{1: array([114508], dtype=int64),
 2: array([  208092,   358273,  6751668,  7131622,  7975244,  7984734,
         8367814,  8579674,  8946378, 10039344], dtype=int64),
 3: array([ 790636, 1800241, 2278871, 2395417, 3344922], dtype=int64),
 4: array([ 267626, 1343092, 1477855, 1920849, 2024432, 2084970], dtype=int64),
 5: array([2884206, 3040964, 5022702], dtype=int64),
 6: array([1800241, 2378281, 2980516], dtype=int64),
 7: array([1001508, 1142988, 1292566, 1355644, 1441953, 1454029, 1878870,
        2250912, 2543164, 3783958], dtype=int64),
 8: array([ 237572,  810819, 1392190, 1663202, 1895587, 2080374, 2381111,
        2494362, 3076658, 3532216, 3659388, 4934950, 5052448, 5657856],
       dtype=int64),
 9: array([ 317740, 3612616], dtype=int64),
 10: array([361748], dtype=int64),
 11: array([ 481141, 1282140, 2388771, 4477536, 5213744, 6450804, 6957966,
        7286456, 8201170, 8579674], dtype=int64),
 12: array([3226786], dtype=int64),
 13: array([1478839, 7343762, 8367814, 85796

`3.` If a user hasn't rated more than 2 movies, we consider these users "too new".  Create a new dictionary that only contains users who have rated more than 2 movies.  This dictionary will be used for all the final steps of this workbook.

In [18]:
# Remove individuals who have watched 2 or fewer movies - don't have enough data to make recs

def create_movies_to_analyze(movies_seen, lower_bound=2):
    '''
    INPUT:  
    movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    lower_bound - (an int) a user must have more movies seen than the lower bound to be added to the movies_to_analyze dictionary

    OUTPUT: 
    movies_to_analyze - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    The movies_seen and movies_to_analyze dictionaries should be the same except that the output dictionary has removed 
    
    '''
    # Implement your code here
    movies_to_analyze = dict()
    for user_id, movies in movies_seen.items():
        if len(movies) > lower_bound:
            movies_to_analyze[user_id] = movies
    return movies_to_analyze

movies_to_analyze = create_movies_to_analyze(movies_seen)

In [19]:
# Run the tests below to check that your movies_to_analyze matches the solution
assert len(movies_to_analyze) == 3494, "Oops!  It doesn't look like your dictionary has the right number of individuals."
assert len(movies_to_analyze[2]) == 10, "Oops!  User 2 didn't match the number of movies we thought they would have."
assert len(movies_to_analyze[7])  == 10, "Oops!  User 7 didn't match the number of movies we thought they would have."
print("If this is all you see, you are good to go!")

If this is all you see, you are good to go!


### Calculating User Similarities

Now that you have set up the **movies_to_analyze** dictionary, it is time to take a closer look at the similarities between users. Below is the pseudocode for how I thought about determining the similarity between users:

```
for user1 in movies_to_analyze
    for user2 in movies_to_analyze
        see how many movies match between the two users
        if more than two movies in common
            pull the overlapping movies
            compute the distance/similarity metric between ratings on the same movies for the two users
            store the users and the distance metric
```

However, this took a very long time to run, and other methods of performing these operations did not fit on the workspace memory!

Therefore, rather than creating a dataframe with all possible pairings of users in our data, your task for this question is to look at a few specific examples of the correlation between ratings given by two users.  For this question consider you want to compute the [correlation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) between users.

`4.` Using the **movies_to_analyze** dictionary and **user_by_movie** dataframe, create a function that computes the correlation between the ratings of similar movies for two users.  Then use your function to compare your results to ours using the tests below.  

In [52]:
def compute_correlation(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    # Implement your code here
    movies1 = movies_to_analyze[user1]
    movies2 = movies_to_analyze[user2]
    sim_movs = np.intersect1d(movies1, movies2, assume_unique = True)
    df = user_by_movie.loc[(user1,user2),sim_movs]
    corr = df.transpose().corr().iloc[0,1]
    return corr #return the correlation

In [56]:
movies1 = movies_to_analyze[8022]
movies1

array([25316, 33467, 34583, 40746, 41959, 43014, 44079, 46359],
      dtype=int64)

In [59]:
movies2 = movies_to_analyze[7]
movies2
sim_movs = np.intersect1d(movies1, movies2, assume_unique = True)
sim_movs

array([], dtype=int64)

In [53]:
type(user_by_movie)

pandas.core.frame.DataFrame

In [48]:
# Test your function against the solution
assert compute_correlation(2,2) == 1.0, "Oops!  The correlation between a user and itself should be 1.0."
assert round(compute_correlation(35,51), 2) == 0.84, "Oops!  The correlation between user 25 and 51 should be about 0.84."
assert np.isnan(compute_correlation(7,8022)), "Oops!  The correlation between user 7 and 8022 should be a NaN."

print("If this is all you see, then it looks like your function passed all of our tests!")

If this is all you see, then it looks like your function passed all of our tests!


0.0

### Why the NaN's?

If the function you wrote passed all of the tests, then you have correctly set up your function to calculate the correlation between any two users.  

`5.` But one question is, why are we still obtaining **NaN** values?  As you can see in the code cell above, users 7 and 8022 have a correlation of **NaN**. Why?

Think and write your ideas here about why these NaNs exist, and use the cells below to do some coding to validate your thoughts. You can check other pairs of users and see that there are actually many NaNs in our data. These NaN's ultimately make the correlation coefficient a less than optimal measure of similarity between two users.

```
In the denominator of the correlation coefficient, we calculate the standard deviation for each user's ratings.  The ratings for two users share the same ratings on the matching movies.  Therefore, the standard deviation is 0.  Because a 0 is in the denominator of the correlation coefficient, we end up with a **NaN** correlation coefficient.  Therefore, a different approach is likely better for this particular situation.
```

In [61]:
# Which movies did both user 2 and user 104 see?
movies1 = set(movies_to_analyze[104])
movies2 = set(movies_to_analyze[7])
movies1.intersection(movies2)

set()

In [62]:
movies1 = set(movies_to_analyze[35])
movies2 = set(movies_to_analyze[51])
movies1.intersection(movies2)

{110475, 137523, 993846, 1234721, 1392190, 1800241, 2788710, 2802144, 3063516}

In [65]:
# What were the ratings for each user on those movies?
print(user_by_movie.loc[35, movies1.intersection(movies2)])
print(user_by_movie.loc[51, movies1.intersection(movies2)])
compute_correlation(35,51)

movie_id
2802144    10.0
1234721     7.0
2788710     9.0
110475      6.0
1800241     7.0
137523      9.0
993846      8.0
3063516     6.0
1392190     9.0
Name: 35, dtype: float64
movie_id
2802144    8.0
1234721    6.0
2788710    7.0
110475     6.0
1800241    7.0
137523     8.0
993846     8.0
3063516    6.0
1392190    8.0
Name: 51, dtype: float64


0.8446877845160873

`6.` Because the correlation coefficient proved to be less than optimal for relating user ratings to one another, we could instead calculate the euclidean distance between the ratings.  I found [this post](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) particularly helpful when I was setting up my function.  This function should be very similar to your previous function.  When you feel confident with your function, test it against our results.

In [68]:
def compute_euclidean_dist(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the euclidean distance between user1 and user2
    '''
    # Implement your code here
    # Pull movies for each user
    movies1 = movies_to_analyze[user1]
    movies2 = movies_to_analyze[user2]
    
    
    # Find Similar Movies
    sim_movs = np.intersect1d(movies1, movies2, assume_unique=True)
    
    # Calculate euclidean distance between the users
    df = user_by_movie.loc[(user1, user2), sim_movs]
    dist = np.linalg.norm(df.loc[user1] - df.loc[user2]) 
    return dist #return the euclidean distance

In [69]:
# Test your function against the solution
assert compute_euclidean_dist(2,2) == 0.0, "Oops!  The distance between a user and itself should be 0.0."
assert round(compute_euclidean_dist(35,51), 2) == 3.32, "Oops!  The distance between user 2 and 66 should be about 2.24."

print("If this is all you see, then it looks like your function passed all of our tests!")

If this is all you see, then it looks like your function passed all of our tests!


In [97]:
compute_euclidean_dist(7,8022)

0.0

In [71]:
# Don't change the code here
# This loads our solution dataframe with every possible user pairings
df_dists = pd.read_csv('df_dists.csv')
del df_dists['Unnamed: 0']
df_dists.dropna(inplace=True)

### Using the Nearest Neighbors to Make Recommendations

In the previous question, you read in **df_dists**. Therefore, you have a measure of distance between each user and every other user. This dataframe holds every possible pairing of users, as well as the corresponding euclidean distance.

Because of the **NaN** values that exist within the correlations of the matching ratings for many pairs of users, as we discussed above, we will proceed using **df_dists**. You will want to find the users that are 'nearest' each user.  Then you will want to find the movies the closest neighbors have liked to recommend to each user.

I made use of the following objects:

* df_dists (to obtain the neighbors)
* user_items (to obtain the movies the neighbors and users have rated)
* movies (to obtain the names of the movies)

`7.` Complete the functions below, which allow you to find the recommendations for any user.  There are five functions which you will need:

* **find_closest_neighbors** - this returns a list of user_ids from closest neighbor to farthest neighbor using euclidean distance


* **movies_liked** - returns an array of movie_ids


* **movie_names** - takes the output of movies_liked and returns a list of movie names associated with the movie_ids


* **make_recommendations** - takes a user id and goes through closest neighbors to return a list of movie names as recommendations


* **all_recommendations** = loops through every user and returns a dictionary of with the key as a user_id and the value as a list of movie recommendations

In [72]:
df_dists.head()

,user1,user2,eucl_dist
8023,2,2,0.0
8024,2,3,0.0
8025,2,4,0.0
8026,2,5,0.0
8027,2,6,0.0


In [76]:
user_items.head()

,user_id,movie_id,rating
0,1,114508,8
1,2,208092,5
2,2,358273,9
3,2,10039344,5
4,2,6751668,9


In [86]:
movies.head()
type(movies)

pandas.core.frame.DataFrame

In [87]:
type(user_items)

pandas.core.frame.DataFrame

In [91]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # Implement your code here
    closest_neighbors = df_dists[df_dists['user1'] == user].sort_values(by = 'eucl_dist').iloc[1:]['user2']
    closest_neighbors = np.array(closest_neighbors)
    return closest_neighbors
    
    
    
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
    movies_liked - an array of movies the user has watched and liked
    '''
    # Implement your code here
    movies_liked = np.array(user_items.query('user_id == @user_id and rating >(@min_rating-1)')['movie_id'])
    return movies_liked


def movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids    
    '''
    # Implement your code here
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
    return movie_lst
    
    
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    # Implement your code here
    # movies_seen by user
    movies_seen = movies_watched(user)
    closest_neighbors = find_closest_neighbors(user)
    
    # keep the recommended movies here
    recs = np.array([])
    # Go through the neighbors and identify movies they like that the user hasn't seen
    for neighbor in closest_neighbors:
        neighbor_like = movies_liked(neighbor)
        new_recs = np.setdiff1d(neighbor_like, movies_seen, assume_unique = True)
        recs = np.unique(np.concatenate([new_recs, recs], axis = 0))
        if len(recs) > num_recs - 1:
            break
    recommendations = movie_names(recs)
    return recommendations

def all_recommendations(num_recs=10):
    '''
    INPUT 
        num_recs (int) the (max) number of recommendations for each user
    OUTPUT
        all_recs - a dictionary where each key is a user_id and the value is an array of recommended movie titles
    '''
    # Implement your code here
    users = np.unique(df_dists['user1'])
    n_users = len(users)
    
    #Store all recommendations in this dict
    all_recs = dict()
    
    #Make the recommendations for each user
    for user in users:
        all_recs[user] = make_recommendations(user, num_recs)
    return all_recs

all_recs = all_recommendations(10)

In [92]:
assert all_recs[2] == make_recommendations(2), "Oops!  Your recommendations for user 2 didn't match ours."
assert all_recs[39] == make_recommendations(39), "Oops!  It actually wasn't possible to make any recommendations for user 39."
assert all_recs[1677] == make_recommendations(1677), "Oops! Looks like your solution for user 1677 didn't match ours."
print("If you made it here, you now have recommendations for many users using collaborative filtering!")
HTML('<img src="images/greatjob.webp">')

If you made it here, you now have recommendations for many users using collaborative filtering!


### Now What?

If you made it this far, you have successfully implemented a solution to making recommendations using collaborative filtering. 

`8.` Let's do a quick recap of the steps taken to obtain recommendations using collaborative filtering.  

In [93]:
# Check your understanding of the results by correctly filling in the dictionary below
a = "pearson's correlation and spearman's correlation"
b = 'item based collaborative filtering'
c = "there were too many ratings to get a stable metric"
d = 'user based collaborative filtering'
e = "euclidean distance and pearson's correlation coefficient"
f = "manhattan distance and euclidean distance"
g = "spearman's correlation and euclidean distance"
h = "the spread in some ratings was zero"
i = 'content based recommendation'

sol_dict = {
    'The type of recommendation system implemented here was a ...':d, # enter a letter,
    'The two methods used to estimate user similarity were: ': e,# enter a letter,
    'There was an issue with using the correlation coefficient.  What was it?': h# enter a letter
}

t.test_recs(sol_dict)

"That's right! All of your solutions look good!"

In [ ]:
# Use the cells below for any work you need to do!

In [94]:
# Users without recs
users_without_recs = []
for user, movie_recs in all_recs.items():
    if len(movie_recs) == 0:
        users_without_recs.append(user)
    
len(users_without_recs)

0

In [ ]:
# NaN euclidean distance values
df_dists['eucl_dist'].isnull().sum()

In [95]:
# Users with fewer than 10 recs
users_with_less_than_10recs = []
for user, movie_recs in all_recs.items():
    if len(movie_recs) < 10:
        users_with_less_than_10recs.append(user)
    
len(users_with_less_than_10recs)

0